In [1]:
# Only execute once: 
# pip install pennylane_qiskit

## Imports

In [3]:
import pennylane as qml
import pennylane_qiskit as pq
from gate_configurations import circuit
import numpy as np
import random

import tensorflow as tf

### -----------------------------------------------------------------------------------------

In [5]:
num_of_qubits = 4
num_of_rot_gates = 4

random_params = [round(random.uniform(0, 2*np.pi),2) for i in range(num_of_rot_gates)]
params_dict = {'w'+str(i): random_params[i] for i in range(num_of_rot_gates)}

circ = circuit(num_of_qubits, params_dict)
circ.build_cascade_rx()
circ.cx_almost_all_neighbors()

print('Param Dict:', circ.get_rotation_params(), '\n')
print(circ.get_quantum_circuit().draw())

Param Dict: {'w0': 1.51, 'w1': 0.82, 'w2': 5.03, 'w3': 1.84} 

     ┌──────────┐               
q_0: ┤ Rx(1.51) ├──■────────────
     ├──────────┤┌─┴─┐          
q_1: ┤ Rx(0.82) ├┤ X ├──■───────
     ├──────────┤└───┘┌─┴─┐     
q_2: ┤ Rx(5.03) ├─────┤ X ├──■──
     ├──────────┤     └───┘┌─┴─┐
q_3: ┤ Rx(1.84) ├──────────┤ X ├
     └──────────┘          └───┘
c: 4/═══════════════════════════
                                


In [29]:
weights = circ.get_rotation_params()
print('Weights:', weights)

Weights: {'w0': 1.51, 'w1': 0.82, 'w2': 5.03, 'w3': 1.84}


In [8]:
qiskit_circ = circ.get_quantum_circuit()
pennylane_circ = pq.load(qiskit_circ)

circ_drawer = qml.draw(pennylane_circ)
print(circ_drawer())

0: ──RX(1.51)─╭●───────┤  
1: ──RX(0.82)─╰X─╭●────┤  
2: ──RX(5.03)────╰X─╭●─┤  
3: ──RX(1.84)───────╰X─┤  


In [25]:
n_qubits = 4

dev = qml.device('default.qubit', wires=n_qubits, shots=1000)
print(dev)

@qml.qnode(dev)
def qnode(inputs, weights):
    print('Inputs  :', inputs)
    print('Weights :', weights)
    
    qml.RX(inputs[0], wires=0)
    qml.RX(inputs[1], wires=1)
    qml.RX(inputs[2], wires=2)
    qml.RX(inputs[3], wires=3)
    
    pennylane_circ(wires=[i for i in range(n_qubits)])
    
    return qml.expval(qml.PauliZ(0)), qml.expval(qml.PauliZ(1)), qml.expval(qml.PauliZ(2)), qml.expval(qml.PauliZ(3))

weight_shapes = {"weights": (2,2)}

Default qubit PennyLane plugin
Short name: default.qubit
Package: pennylane
Plugin version: 0.29.1
Author: Xanadu Inc.
Wires: 4
Shots: 1000


In [26]:
circuit = qml.QNode(qnode, dev)
qlayer = qml.qnn.KerasLayer(circuit, weight_shapes, output_dim=4)

In [27]:
qlayer.get_config()

{'name': 'keras_layer_2',
 'trainable': True,
 'dtype': 'float32',
 'dynamic': True,
 'output_dim': 8,
 'weight_specs': {},
 'weight_shapes': {'weights': (2, 2)},
 'argnum': None}

In [28]:
print(qlayer.qnode([0,0,0,0], [0.25, 0.56, 0.75, 1]))

Inputs  : [0, 0, 0, 0]
Weights : [0.25, 0.56, 0.75, 1]
tf.Tensor([0.134 0.05  0.04  0.022], shape=(4,), dtype=float64)


### You must execute the following code to convert a QISKIT circuit into a Pennylane quantum circuit. Look at the example above for more details. 

```
qiskit_circ = qc.get_quantum_circuit()

pennylane_circ = pq.load(qiskit_circ)

# To visualize only
circ_drawer = qml.draw(pennylane_circ)
print(circ_drawer)
```